In [15]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set Plotly renderer
pio.renderers.default = "iframe"

# --- Extract Tesla Stock Data ---
tesla_data = yf.download("TSLA", start="2010-01-01", end="2021-06-30")
tesla_data.reset_index(inplace=True)

# --- Extract Tesla Revenue Data ---
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

tables = soup.find_all("table")
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        for row in table.find("tbody").find_all("tr"):
            cols = row.find_all("td")
            if len(cols) == 2:
                date = cols[0].text.strip()
                revenue = cols[1].text.strip()
                tesla_revenue = pd.concat(
                    [tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})],
                    ignore_index=True
                )

/tmp/ipykernel_1238/97074070.py:13: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [16]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [17]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data  = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
tables = soup.find_all("table")
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        # Iterate through the table rows and extract data
        for row in table.find("tbody").find_all("tr"):
            cols = row.find_all("td")
            if len(cols) == 2:
                date = cols[0].text.strip()
                revenue = cols[1].text.strip()
                # Append each row to the DataFrame
                tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})],
                                          ignore_index=True)
# Display the last few rows
tesla_revenue.tail()
# Reset the index of tesla_data
tesla_data.reset_index(inplace=True)

In [18]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
tables = soup.find_all("table")
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        # Iterate through the table rows and extract data
        for row in table.find("tbody").find_all("tr"):
            cols = row.find_all("td")
            if len(cols) == 2:
                date = cols[0].text.strip()
                revenue = cols[1].text.strip()
                # Append each row to the DataFrame
                tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})],
                                          ignore_index=True)
                

In [21]:
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,31
49,2010-06-30,28
50,2010-03-31,21
52,2009-09-30,46
53,2009-06-30,27


In [19]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"",regex=True)
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [20]:
import plotly.graph_objects as go
import pandas as pd

# Define the make_graph function
def make_graph(stock_data, revenue_data, stock_name):
    fig = go.Figure()

    # Add stock price line
    fig.add_trace(go.Scatter(
        x=stock_data["Date"],
        y=stock_data["Close"],
        name="Stock Price"
    ))

    # Add revenue line if revenue data is provided
    if revenue_data is not None and not revenue_data.empty:
        fig.add_trace(go.Scatter(
            x=revenue_data["Date"],
            y=revenue_data["Revenue"],
            name="Revenue"
        ))

    # Layout
    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue (Up to June 2021)",
        xaxis_title="Date",
        yaxis_title="Price / Revenue (USD)",
        hovermode="x unified"
    )

    fig.show()
# --- Clean and Filter Data ---
# Clean Revenue column 
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r"[\$,]", "", regex=True) 
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce") 

# Filter both datasets to show data up to June 2021 
tesla_data = tesla_data[tesla_data["Date"] <= "2021-06-30"] 
tesla_revenue = tesla_revenue[tesla_revenue["Date"] <= "2021-06-30"] 
# --- Create the Graph ---
make_graph(tesla_data, tesla_revenue, "Tesla")